In [9]:
import docx2txt
import itertools
import pandas as pd
from difflib import SequenceMatcher
import re
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [14]:
text = docx2txt.process("MY COPY_Final HLA.P.1631_Adenda do EIA do Bloco 17_20200620 - Copy EN (1) - Copy.docx")

In [15]:
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences


In [16]:
sents = split_into_sentences(text)
sents

['7  \t\t  \t\t  \t\t  \t\t  \t\tAddendum to the Environmental Impact Assessment for the Block 17 Seismic Acquisition and Drilling Program  \t\tFinal Considerations    \t\t  \t\t  \t\t  \t\t  \t\t  \t\t  \t\t  \t\t8  \t\t  \t\t  \t\t  \t\t  \t\tAddendum to the Environmental Impact Assessment for the Block 17 Seismic Acquisition and Drilling Program  \t\tBibliography    \t\t  \t\t  \t\t  \t\t  \t\t  \t\t  \t\t  \t\t6  \t\t  \t\t  \t\t  \t\t  \t\tAddendum to the Environmental Impact Assessment for the Block 17 Seismic Acquisition and Drilling Program  \t\tENVIRONMENTAL MANAGEMENT SYSTEM    \t\t  \t\t  \t\t  \t\t  \t\t  \t\t  \t\t  \t\t6  \t\t  \t\t  \t\t  \t\t  \t\tAddendum to the Environmental Impact Assessment for the Block 17 Seismic Acquisition and Drilling Program  \t\tENVIRONMENTAL MANAGEMENT SYSTEM    \t\t  \t\t  \t\t  \t\t  \t\t  \t\t  \t\t  \t\t5  \t\t  \t\t  \t\t  \t\t  \t\tAddendum to the Environmental Impact Assessment for the Block 17 Seismic Acquisition and Drilling Program

In [17]:
def similar(x, y):
    return SequenceMatcher(None, x, y).ratio()

x_list = []
y_list = []
score = []

for x,y in itertools.combinations(sents, 2):
    fuzz.ratio(x, y)
    score.append(fuzz.ratio(x, y))
    x_list.append(x)
    y_list.append(y)
    
data_tuples = list(zip(x_list,y_list,score))
results = pd.DataFrame(data_tuples, columns=['X','Y', 'Score'])  
results = results.sort_values(by=['Score'], ascending=False)
results

,X,Y,Score
0,7 \t\t \t\t \t\t \t\t \t\tAddendum to the...,Photo Author: TOTAL \tProject: P1631 \t Holí...,4
1,7 \t\t \t\t \t\t \t\t \t\tAddendum to the...,"60th Street, House 559 \tUrbanization Harmony...",3
2,7 \t\t \t\t \t\t \t\t \t\tAddendum to the...,co.,0
3,7 \t\t \t\t \t\t \t\t \t\tAddendum to the...,ao \t CONTENTS Part 1 Introduction\t7 1.,2
4,7 \t\t \t\t \t\t \t\t \t\tAddendum to the...,1 Objectives of the Addendum\t7 1.,2
...,...,...,...
1890535,remss.com www.,totalepangola-mango.,17
1890536,remss.com www.,fr www.,64
1890537,undp.org www.,totalepangola-mango.,12
1890538,undp.org www.,fr www.,61


In [20]:
results = results.sort_values(by=['Score'], ascending=False)

In [24]:
results[results['Score'] > 70].to_csv('results.csv')